In [1]:
import pandas as pd
import numpy as np

# Procesamiento columna 'Descripción'

In [2]:
excel_file = pd.ExcelFile('../data/raw/data.xlsx')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [3]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('../data/raw/data.xlsx', sheet_name=name, converters=converter_descripcion)

1. Separar cada fila en cada una de las descripciones. i.e : "realizar analisis dga, humedad y furanos" deben quedar dos filas exactamente iguales pero con cada uno de los registros. -> pd.explode(), explode a esa accion de expandir un dataset con base a una regla.

2. Aplicar un algoritmo de distancia de levehnstein. 

3. Análisis descriptivo que relacione cada equipo con las acciones. i.e: frecuencias de cada accion para cada equipo.

## Separación manual

In [25]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'] == originales[i]
        if sum(index) > 0:
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df = df.append(instancias, ignore_index = True)
            df.loc[len(df) - sum(index):,'Descripción'] = adiciones[i]
    return df




df_collection_separados = {}

excel_file_reemplazos = pd.ExcelFile('../data/reemplazos_manual/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names
df_collection_manuales = {}

converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

for name in sheets_reemplazos:#sheets_reemplazos:
    df_collection_manuales[name] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

## Aplicando el algoritmo de distancia de Levenshtein

### Algoritmo de Levenshtein

In [55]:
def dist_lev(s1,s2):
    dist_matrix = np.zeros((len(s1) + 1, len(s2) + 1))
    dist_matrix[1:,0] = range(1,len(s1) + 1)
    dist_matrix[0,1:] = range(1,len(s2) + 1)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1], 
                                    dist_matrix[i, j - 1] + 1)
            else:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1] + 1, 
                                    dist_matrix[i, j - 1] + 1)
    return dist_matrix[-1,-1]

1.0

### Aplicando y ordenando según distancia

In [73]:
# Extrayendo valores únicos:
unicos_descripcion = {}
for name in sheets_reemplazos:
    unicos = pd.unique(df_collection_separados[name]['Descripción'])
    unicos_descripcion[name] = [unicos.tolist(), np.zeros((len(unicos),1))]

unicos_descripcion['T-PAP(C)_T-ACE']

[['realizar analisis dga, humedad y furanos',
  'realizar analisis dga para seguimento',
  'realizar analisis dga y furanos',
  'revisar motoventiladores.',
  'revisar control ventiladores atr2 fa',
  'muestra de aceite',
  'revisar indic temperat aceite  atr2- fa',
  'revisar motoventilador t-ctg05',
  'realizar muestreo y analisis dga',
  'revisar breaker ventiladores t-ctg05',
  'cambiar soportes motoventilador t-ctg05',
  '01-realizar muestreo y analisis dga',
  'cambiar motoventilador por alta temp.',
  'cambio motoventiladores t-ctg05',
  'realizar muestreo analisis dga',
  'cambiar ventilador con vibracion alta',
  'reparar ventilador n2 atr-1 fase c',
  'calibrar indicador temperatura devanados',
  'calibrar  indicador temperatura devanado',
  'repar sistema de refrigeracion  fase c',
  'revisar indicacion nivel aceite oltc fb',
  'revisar borneras ventiladores t- ctg05',
  'reparar motoventilador',
  'realizar analsisis de metanol en doble',
  'revisar ruido anormal motovent t